# Azure AI Agent service - Connected agents

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import datetime
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import BingGroundingTool, ConnectedAgentTool, FilePurpose, FileSearchTool, ListSortOrder, MessageRole
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 17-Jun-2025 08:59:53


## Settings

In [4]:
load_dotenv("azure.env")

True

In [5]:
endpoint = os.getenv("PROJECT_ENDPOINT")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

In [6]:
print(dir(project_client))

['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_client', '_config', '_delete_agent', '_deserialize', '_function_tool', '_function_tool_max_retry', '_is_protocol', '_serialize', '_validate_tools_and_tool_resources', 'close', 'create_agent', 'create_thread_and_process_run', 'create_thread_and_run', 'delete_agent', 'enable_auto_function_calls', 'files', 'get_agent', 'list_agents', 'messages', 'run_steps', 'runs', 'send_request', 'threads', 'update_agent', 'vector_store_file_batches', 'vector_store_files', 'vector_stores']


## Agents definition

### 1. Stocks price agent

In [7]:
bing = BingGroundingTool(connection_id=os.getenv("BING_CONN_ID"))

In [8]:
stocksprice_agent = project_client.create_agent(
    model="gpt-4o",
    name="stocks-price-agent-demo",
    instructions="Retrieve the current stock price for a specified company.",
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"},
    description="Stocks price agent (demo)",
    temperature=0.7,
)

In [9]:
print(f"Created agent, ID: {stocksprice_agent.id}")

Created agent, ID: asst_DgAzZkR7EfvOo6tOpKgt2YJi


### 2. Currency agent

In [10]:
currency_convert_agent = project_client.create_agent(
    model="gpt-4o",
    name="currency-convert-agent-demo",
    instructions="Retrieve the current currency exchange rate.",
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"},
    description="Currency convert agent (demo)",
    temperature=0.7,
)

In [11]:
print(f"Created agent, ID: {currency_convert_agent.id}")

Created agent, ID: asst_2vK1ub32tjOLoyCRXnv6xcpG


## 3. RAG agent

In [12]:
DATA_DIR = "data"

pdf_file = os.path.join(DATA_DIR, "2024_Annual_Report.pdf")

In [13]:
!ls $pdf_file -lh

-rwxrwxrwx 1 root root 1.2M Jun 17 07:35 data/2024_Annual_Report.pdf


In [14]:
file = project_client.files.upload_and_poll(file_path=pdf_file,
                                                  purpose=FilePurpose.AGENTS)

print(f"Uploaded file, file ID: {file.id}")

# create a vector store with the file you uploaded
vector_store = project_client.vector_stores.create_and_poll(
    file_ids=[file.id], name="document_vector_store")

print(f"Created vector store, vector store ID: {vector_store.id}")

Uploaded file, file ID: assistant-DMN9MtsptJ8H4bqFaaPA6R
Created vector store, vector store ID: vs_WvWdFCSvoILl6fbDubrU57yS


In [15]:
# create a file search tool
file_search_tool = FileSearchTool(vector_store_ids=[vector_store.id])

# notices that FileSearchTool as tool and tool_resources must be added or the agent will be unable to search the file
document_pdf_agent = project_client.create_agent(
    model="gpt-4o",
    name="document-agent-pdf-demo",
    instructions="You are an AI helpful agent to analyse this document which is the Microsoft 2024 annual report",
    tools=file_search_tool.definitions,
    tool_resources=file_search_tool.resources,
    description="PDF document agent (demo)",
    temperature=0.7,
)

print(f"Created agent, agent ID: {document_pdf_agent.id}")

Created agent, agent ID: asst_fXuytFFtqkXnUYbJkBOkRMnO


### 3. Main agent with connected agents

In [16]:
connected_agent_stocks = ConnectedAgentTool(
    id=stocksprice_agent.id,
    name="stockspriceagentconnection",
    description="Get the stock price of a company")

In [17]:
connected_agent_currency = ConnectedAgentTool(
    id=currency_convert_agent.id,
    name="currencyconvertagentconnection",
    description="Convert an amount into another currency")

In [18]:
connected_document = ConnectedAgentTool(
    id=document_pdf_agent.id,
    name="documentagentconnection",
    description="You are an AI helpful agent to analyse this document which is the Microsoft 2024 annual report")

In [19]:
main_agent_demo = project_client.create_agent(
    model="gpt-4o",
    name="main-agent-demo",
    instructions="You are the main AI agent",
    description="Main agent (demo)",
    tools=[
        connected_agent_stocks.definitions[0],
        connected_agent_currency.definitions[0],
        connected_document.definitions[0],
    ],
)

In [20]:
print(f"Created agent, ID: {main_agent_demo.id}")

Created agent, ID: asst_xqgm6DqTi8p6KpxSER8rnZGD


## List of agents

In [21]:
agents_list = [stocksprice_agent, currency_convert_agent, document_pdf_agent, main_agent_demo]

for agent in agents_list:
    print(
        f"Agent Details:\n"
        f"📛 Name        : {agent.name}\n"
        f"🆔 ID          : {agent.id}\n"
        f"🧠 Model       : {agent.model}\n"
        f"📜 Instructions: {agent.instructions}\n"
    )

Agent Details:
📛 Name        : stocks-price-agent-demo
🆔 ID          : asst_DgAzZkR7EfvOo6tOpKgt2YJi
🧠 Model       : gpt-4o
📜 Instructions: Retrieve the current stock price for a specified company.

Agent Details:
📛 Name        : currency-convert-agent-demo
🆔 ID          : asst_2vK1ub32tjOLoyCRXnv6xcpG
🧠 Model       : gpt-4o
📜 Instructions: Retrieve the current currency exchange rate.

Agent Details:
📛 Name        : document-agent-pdf-demo
🆔 ID          : asst_fXuytFFtqkXnUYbJkBOkRMnO
🧠 Model       : gpt-4o
📜 Instructions: You are an AI helpful agent to analyse this document which is the Microsoft 2024 annual report

Agent Details:
📛 Name        : main-agent-demo
🆔 ID          : asst_xqgm6DqTi8p6KpxSER8rnZGD
🧠 Model       : gpt-4o
📜 Instructions: You are the main AI agent



### Quick test

In [22]:
prompt = "Hello."

In [23]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_rK6e5g3IZVTcnGh1guo6Bwgg
💬 [2] Created message | ID: msg_p7XYZuztlhqgUF5hAfCqoqMU
🏃 [3] Created run     | ID: run_XEbSQt671wJXjtWv6uRI8rtR
⏳ [4] Polling run status...
🔄 Status: completed

🔍 Step ID details: step_dfZF2GebSUDU9zKoDiqGKegl, Type: message_creation
Details step: {'id': 'step_dfZF2GebSUDU9zKoDiqGKegl', 'object': 'thread.run.step', 'created_at': 1750150810, 'run_id': 'run_XEbSQt671wJXjtWv6uRI8rtR', 'assistant_id': 'asst_xqgm6DqTi8p6KpxSER8rnZGD', 'thread_id': 'thread_rK6e5g3IZVTcnGh1guo6Bwgg', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750150811, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_mSU4X1Gx8CcpiGTIzjnLHqhG'}}, 'usage': {'prompt_tokens': 398, 'completion_tokens': 11, 'total_tokens': 409, 'prompt_token_details': {'cached_tokens': 0}}}

🤖 [5] Agent responses:

💡 → user: Hello.

💡 → assistant:

In [24]:
run

{'id': 'run_XEbSQt671wJXjtWv6uRI8rtR', 'object': 'thread.run', 'created_at': 1750150810, 'assistant_id': 'asst_xqgm6DqTi8p6KpxSER8rnZGD', 'thread_id': 'thread_rK6e5g3IZVTcnGh1guo6Bwgg', 'status': 'completed', 'started_at': 1750150810, 'expires_at': None, 'cancelled_at': None, 'failed_at': None, 'completed_at': 1750150811, 'required_action': None, 'last_error': None, 'model': 'gpt-4o', 'instructions': 'You are the main AI agent', 'tools': [{'type': 'connected_agent', 'connected_agent': {'name': 'stockspriceagentconnection', 'description': 'Get the stock price of a company', 'id': 'asst_DgAzZkR7EfvOo6tOpKgt2YJi'}}, {'type': 'connected_agent', 'connected_agent': {'name': 'currencyconvertagentconnection', 'description': 'Convert an amount into another currency', 'id': 'asst_2vK1ub32tjOLoyCRXnv6xcpG'}}, {'type': 'connected_agent', 'connected_agent': {'name': 'documentagentconnection', 'description': 'You are an AI helpful agent to analyse this document which is the Microsoft 2024 annual rep

In [25]:
run.usage

{'prompt_tokens': 398, 'completion_tokens': 11, 'total_tokens': 409, 'prompt_token_details': {'cached_tokens': 0}}

### Example 1

In [26]:
prompt = "What is microsoft's share price?"

In [27]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_6u1lXLOnLzO4bnCg1Jy3DHGb
💬 [2] Created message | ID: msg_P3o5qz63Ob59yiAEQdcZVuHg
🏃 [3] Created run     | ID: run_b0EL5ev0xsWR3L5i6JQrqkiV
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🔍 Step ID details: step_U6L5BAWn4AN8iEstgRxBQbjS, Type: message_creation
Details step: {'id': 'step_U6L5BAWn4AN8iEstgRxBQbjS', 'object': 'thread.run.step', 'created_at': 1750150820, 'run_id': 'run_b0EL5ev0xsWR3L5i6JQrqkiV', 'assistant_id': 'asst_xqgm6DqTi8p6KpxSER8rnZGD', 'thread_id': 'thread_6u1lXLOnLzO4bnCg1Jy3DHGb', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750150821, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_97NSLsDGwABPWCvYD5HQ8cYI'}}, 'usage': {'prompt_tokens': 485, 'completion_tokens': 38, 'total_tokens':

### Example 2

In [28]:
prompt = "Convert this into € and in Japanese Yen"

In [29]:
# Step 1: Use a thread
#thread = project_client.threads.create()
print(f"🧵 [1] Using existing thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Using existing thread  | ID: thread_6u1lXLOnLzO4bnCg1Jy3DHGb
💬 [2] Created message | ID: msg_wddCvYoW11n7bWWuzk9LD9Zo
🏃 [3] Created run     | ID: run_FN7XFjtksKPO01hwlanYI2uy
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🔍 Step ID details: step_6AKHq0stcOYsfxSxkjfIty2N, Type: message_creation
Details step: {'id': 'step_6AKHq0stcOYsfxSxkjfIty2N', 'object': 'thread.run.step', 'created_at': 1750150832, 'run_id': 'run_FN7XFjtksKPO01hwlanYI2uy', 'assistant_id': 'asst_xqgm6DqTi8p6KpxSER8rnZGD', 'thread_id': 'thread_6u1lXLOnLzO4bnCg1Jy3DHGb', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750150833, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_QCjNBTsN2oSS8GKiwABrI0tH'}}, 'usage': {'prompt_tokens': 690, 'completion

### Example 3

In [30]:
prompt = "I have 10 Microsoft shares + 5 IBM and 7 SFDC shares. What is the total value of my portfolio in eur?"

In [31]:
# Step 1: Create a new thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_pUjOjOKQSisMXIDC231OdCfi
💬 [2] Created message | ID: msg_KIB8JgOyWbZ7tTqky04cv6bI
🏃 [3] Created run     | ID: run_RD9sbAqi0BJAdabkKFdSLYX6
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🔍 Step ID details: step_cdwZuKxMubEorSILjjaeyMQ2, Type: message_creation
Details step: {'id': 'step_cdwZuKxMubEorSILjjaeyMQ2', 'object': 'thread.run.step', 'created_at': 1750150854, 'run_id': 'run_RD9sbAqi0BJAdabkKFdSLYX6', 'assistant_id': 'asst_xqgm6DqTi8p6KpxSER8rnZGD', 'thread_id': 'thread_pUjOjOKQSisMXIDC231OdCfi', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750150856, 'expires_at': None, 'fail

### Example 4

In [32]:
prompt = "What are the highlights for Microsoft from fiscal year 2024 compared with fiscal year 2023?"

In [33]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_zJGm7HWUiyjMPcBzJGev4wq9
💬 [2] Created message | ID: msg_hBbwmiTOmbBGcaUY90TLqCXm
🏃 [3] Created run     | ID: run_8LikQ8OlIRFw8AfMrA2H8DfG
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🔍 Step ID details: step_ueMoqmm1hg5YyDIkwZGpx6tg, Type: message_creation
Details step: {'id': 'step_ueMoqmm1hg5YyDIkwZGpx6tg', 'object': 'thread.run.step', 'created_at': 1750150872, 'run_id': 'run_8LikQ8OlIRFw8AfMrA2H8DfG', 'assistant_id': 'asst_xqgm6DqTi8p6KpxSER8rnZGD', 'thread_id': 'thread_zJGm7HWUiyjMPcBzJGev4wq9', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750150878, 'expires_at': None, 'failed_at': None, 'last_er

### Example 5

In [34]:
prompt = "What are the main products and companies mentionned in the 2024 Microsoft annual report?"

In [35]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread  | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"🏃 [3] Created run     | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

run_steps = project_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"\n🔍 Step ID details: {step.id}, Type: {step.type}")
    print(f"Details step: {step}")

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"💡 → {msg.role}: {last_text.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        idx = 1
        for annotation in msg.url_citation_annotations:
            print(f"🔗 Source {idx}: [{annotation.url_citation.title}]({annotation.url_citation.url})")
            idx += 1

print("\033[0m")  # Reset color

🧵 [1] Created thread  | ID: thread_TuVm0ZHcu6NgryqfetbGnTnh
💬 [2] Created message | ID: msg_vecpbLK9txReMFJ1csqNRIGM
🏃 [3] Created run     | ID: run_LBR88IqPDjee9ZpSbNauywjB
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🔍 Step ID details: step_Lq2zM2ipmQNtLZ8hc2uAvx99, Type: message_creation
Details step: {'id': 'step_Lq2zM2ipmQNtLZ8hc2uAvx99', 'object': 'thread.run.step', 'created_at': 1750150893, 'run_id': 'run_LBR88IqPDjee9ZpSbNauywjB', 'assistant_id': 'asst_xqgm6DqTi8p6KpxSER8rnZGD', 'thread_id': 'thread_TuVm0ZHcu6NgryqfetbGnTnh', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1750150897, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'messag

## Post processing

In [36]:
agents_list = [stocksprice_agent, currency_convert_agent, document_pdf_agent, main_agent_demo]

for agent in agents_list:
    print(
        f"Agent Details:\n"
        f"📛 Name        : {agent.name}\n"
        f"🆔 ID          : {agent.id}\n"
        f"🧠 Model       : {agent.model}\n"
        f"📜 Instructions: {agent.instructions}\n"
    )

Agent Details:
📛 Name        : stocks-price-agent-demo
🆔 ID          : asst_DgAzZkR7EfvOo6tOpKgt2YJi
🧠 Model       : gpt-4o
📜 Instructions: Retrieve the current stock price for a specified company.

Agent Details:
📛 Name        : currency-convert-agent-demo
🆔 ID          : asst_2vK1ub32tjOLoyCRXnv6xcpG
🧠 Model       : gpt-4o
📜 Instructions: Retrieve the current currency exchange rate.

Agent Details:
📛 Name        : document-agent-pdf-demo
🆔 ID          : asst_fXuytFFtqkXnUYbJkBOkRMnO
🧠 Model       : gpt-4o
📜 Instructions: You are an AI helpful agent to analyse this document which is the Microsoft 2024 annual report

Agent Details:
📛 Name        : main-agent-demo
🆔 ID          : asst_xqgm6DqTi8p6KpxSER8rnZGD
🧠 Model       : gpt-4o
📜 Instructions: You are the main AI agent



In [37]:
#for agent in agents_list:
#    project_client.delete_agent(agent.id)
#    print(f"Deleted agent, agent ID: {agent.id}")